In [1]:
using CairoMakie, Dierckx, QuantEcon, LinearAlgebra, Optim

In [8]:
pars = (;α = 0.33, # capital share
    β = 0.9, # discount factor
    γ = 1.0, # risk aversion
    nz = 19, # number of grid points for z
    ρ = 0.9, # persistence of AR1
    μ = 0.0, # mean of AR1
    σ = 0.1, # std of AR1
    nk = 31, # number of capital grid points
    θ = 3.0) # grid expansion factor

(α = 0.33, β = 0.9, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.1, nk = 31, θ = 3.0)

In [12]:
function utility(c, pars)
    (; γ) = pars
    if γ == 1.0
        return log(c)
    else
        return c^(1-γ)/(1-γ)
    end
end

function ar1(pars)
    (;ρ, μ, σ, nz) = pars
    mc = QuantEcon.rouwenhorst(nz, μ, ρ, σ)
    return mc.p, mc.state_values
end

function exp_grid(pars)
    (; nk, θ) = pars
    grid = LinRange(0.0,1.0,nk)
    exp_grid = grid_min + (grid_max - grid_min) * grid.^θ
    return exp_grid
end

exp_grid (generic function with 1 method)